# Select row for testing the api first:

In [ ]:
import pandas as pd

df = pd.read_csv("main_dataset10k.csv")

In [ ]:
df.head()

,input
0,INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To ...
1,To the Board of Directors and Shareowners of A...
2,. To the Board of Directors and Stockholders o...
3,. Report of Independent Registered Public Acco...
4,. MGE Energy Management's Report on Internal C...


In [ ]:
# Select the first 3 rows
df_first_3 = df.head(3)

# Save to a CSV file
df_first_3.to_csv("first_3_rows.csv", index=False)

In [ ]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 5.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai
print(openai.__version__)

1.61.0


# GPT_Testing

https://www.analyticsvidhya.com/blog/2023/05/how-to-use-chatgpt-api-in-python/

https://dorik.com/blog/how-to-use-chatgpt-api

reads a CSV file containing financial statements,

extracts important financial sections such as revenue, profit/loss, and expenses using regular expressions, and summarizes the key information using OpenAI's GPT-3.5 model.

also :
* Prepares data for processing
* Loads any previous progress
* Defines functions for extracting and summarizing key sections
* Iterates through the data to generate summaries.
* It periodically saves progress and handles interruptions gracefully
* Saving the summarized financial statements to a CSV file.

In [ ]:
import openai
import pandas as pd
import os
import time
import re

df = pd.read_csv("first_3_rows.csv")

# Define local save path
local_path = "./"
os.makedirs(local_path, exist_ok=True)

save_path = os.path.join(local_path, "financial_statements_summary_safe.csv")  # Output CSV

# Define API key directly
API =
client = openai.OpenAI(api_key=API)

# Define save path
save_path = "financial_statements_summary_safe.csv"

# Keep only the required column
df = df[["input"]].copy()
df["summary"] = ""

# Load previous progress if available
try:
    df_saved = pd.read_csv(save_path)
    df.update(df_saved)
    print("Resumed from saved progress.")
except FileNotFoundError:
    print("No previous save found, starting fresh.")

# Function to extract important financial sections
def extract_important_sections(text):
    sections = []

    # Extract revenue-related sections
    revenue_match = re.search(r"(Revenue|Total Revenue|Net Revenue).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    if revenue_match:
        sections.append("Revenue: " + revenue_match.group())

    # Extract profit/loss-related sections
    profit_match = re.search(r"(Net Income|Net Profit|Loss).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    if profit_match:
        sections.append("Profit/Loss: " + profit_match.group())

    # Extract expenses-related sections
    expenses_match = re.search(r"(Operating Expenses|Total Expenses).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    if expenses_match:
        sections.append("Expenses: " + expenses_match.group())

    # Join extracted parts and return
    return "\n".join(sections) if sections else text[:6000]  # Default to first 6000 words if no match

# Function to summarize text with GPT-3.5
def summarize_text(text):
    important_text = extract_important_sections(text)  # Extract key financial data before summarizing

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Summarize financial reports concisely."},
                {"role": "user", "content": f"Summarize this financial statement:\n\n{important_text}"}
            ],
            max_tokens=150,
            temperature=0.3
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error: {e}")
        return None

# Process and summarize
for i, row in df.iterrows():
    if pd.notna(row["summary"]) and row["summary"] != "":
        continue

    print(f"Processing row {i+1}/{len(df)}...")

    summary = summarize_text(row["input"])
    if summary is None:
        print("API error detected. Stopping.")
        break

    df.at[i, "summary"] = summary

    if i % 10 == 0:
        df.to_csv(save_path, index=False)
        print(f"Progress saved at row {i}")

# Final save
df.to_csv(save_path, index=False)
print("Summarization completed. File saved as:", save_path)

Resumed from saved progress.
Processing row 2/3...
Processing row 3/3...
Summarization completed. File saved as: financial_statements_summary_safe.csv


In [ ]:
print("Input Column:\n", df.iloc[0]["input"])  # Print full input text
print("\nSummary Column:\n", df.iloc[0]["summary"])  # Print full summary

Input Column:
 INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To the Board of Directors and Stockholders Digerati Technologies, Inc. San Antonio, Texas We have audited the accompanying consolidated balance sheets of Digerati Technologies, Inc. and its subsidiaries (collectively, “Digerati”) as of July 31, 2016 and July 31, 2015, and the related consolidated statements of operations, stockholders’ deficit and cash flows for the years then ended. These consolidated financial statements are the responsibility of Digerati’s management. Our responsibility is to express an opinion on these consolidated financial statements based on our audits. We conducted our audits in accordance with standards of the Public Company Accounting Oversight Board (United States). Those standards require that we plan and perform the audits to obtain reasonable assurance about whether the financial statements are free of material misstatement. Digerati is not required to have, nor were we engaged to perform, an audit

# Claude_Testing

https://cubed.run/blog/how-to-get-started-with-claude-api

https://dev.to/engineerdan/generating-python-code-using-anthropic-api-for-claude-ai-4ma4

In [ ]:
pip install anthropic

Note: you may need to restart the kernel to use updated packages.


let's check which model that we can use for our tasks:

In [ ]:
import anthropic

client = anthropic.Anthropic(
    api_key=
)

models = client.models.list()
print(models)


SyncPage[ModelInfo](data=[ModelInfo(id='claude-3-5-sonnet-20241022', created_at=datetime.datetime(2024, 10, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3.5 Sonnet (New)', type='model'), ModelInfo(id='claude-3-5-haiku-20241022', created_at=datetime.datetime(2024, 10, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3.5 Haiku', type='model'), ModelInfo(id='claude-3-5-sonnet-20240620', created_at=datetime.datetime(2024, 6, 20, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3.5 Sonnet (Old)', type='model'), ModelInfo(id='claude-3-haiku-20240307', created_at=datetime.datetime(2024, 3, 7, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3 Haiku', type='model'), ModelInfo(id='claude-3-opus-20240229', created_at=datetime.datetime(2024, 2, 29, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude 3 Opus', type='model')], has_more=False, first_id='claude-3-5-sonnet-20241022', last_id='claude-3-opus-20240229')


https://www.anthropic.com/pricing#anthropic-api

let's go with claude-3-5-haiku-20241022 due to it's cheaper

In [ ]:
import anthropic
import pandas as pd
import os
import time
import re

# Load the first 3 rows from the dataset
df = pd.read_csv("first_3_rows.csv")

# Define save path
save_path = "test_claude_summary.csv"

# Define Claude API key
CLAUDE_API_KEY =

# Initialize Claude client
client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

# Keep only the required column
df = df[["input"]].copy()
df["summary"] = ""

# Function to extract important financial sections
def extract_important_sections(text):
    sections = []

    # Extract revenue-related sections
    revenue_match = re.search(r"(Revenue|Total Revenue|Net Revenue).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    if revenue_match:
        sections.append("Revenue: " + revenue_match.group())

    # Extract profit/loss-related sections
    profit_match = re.search(r"(Net Income|Net Profit|Loss).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    if profit_match:
        sections.append("Profit/Loss: " + profit_match.group())

    # Extract expenses-related sections
    expenses_match = re.search(r"(Operating Expenses|Total Expenses).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    if expenses_match:
        sections.append("Expenses: " + expenses_match.group())

    return "\n".join(sections) if sections else text[:6000]  # Default to first 6000 chars if no match

# Function to summarize text using Claude API
def summarize_text_claude(text):
    important_text = extract_important_sections(text)

    try:
        response = client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=150,
            temperature=0.3,
            messages=[{"role": "user", "content": f"Summarize this financial statement:\n\n{important_text}"}]
        )

        # Debug: Print full API response
        print("Claude API Response:", response)

        # Extract the correct content
        return response.content[0].text.strip()

    except anthropic.RateLimitError as e:
        print(f"Claude rate limit exceeded: {e}. Retrying in 60 seconds...")
        time.sleep(60)
        return summarize_text_claude(text)  # Retry

    except anthropic.APIConnectionError as e:
        print(f"Claude API connection error: {e}. Retrying in 30 seconds...")
        time.sleep(30)
        return summarize_text_claude(text)  # Retry

    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

# Process and summarize only 3 rows
for i, row in df.iterrows():
    if pd.notna(row["summary"]) and row["summary"] != "":
        continue  # Skip already processed rows

    print(f"Processing row {i+1}/{len(df)} with Claude API...")

    summary = summarize_text_claude(row["input"])

    if summary is None:
        print("API error detected. Stopping to avoid data loss.")
        break

    df.at[i, "summary"] = summary

    # Save progress after each row
    df.to_csv("test_claude_summary.csv", index=False)
    print(f"Progress saved at row {i+1}")

# Final save
df.to_csv("test_claude_summary.csv", index=False)
print("\nTest Completed! Summarized results saved in: test_claude_summary.csv")

# Print the summarized results
print("\nClaude API Summaries:")
print(df[["input", "summary"]])


Processing row 1/3 with Claude API...
Claude API Response: Message(id='msg_017FQqYhea1fiFxHV2an4W6R', content=[TextBlock(citations=None, text="Here's a summary of the financial statement:\n\nCompany: Digerati Technologies\n\nRevenue Sources:\n1. Global VoIP Services: Provides communication services to telecommunications companies for international traffic termination\n2. Cloud Communication Services: Offers hosted IP/PBX services, call center applications, and various communication features\n\nRevenue Recognition Principles:\n- Revenue recognized when:\n  * Arrangement evidence exists\n  * Services provided\n  * Price is fixed\n  * Collectability is assured\n  * No significant obligations remain\n\nCost Structures:\n- Global VoIP Services: Transmission charges, network costs, bandwidth fees\n- Cloud Communication Services: Bandwidth, licensing, and co-location charges", type='text')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='max_tokens', stop_sequence=None, typ

In [ ]:
print("Input Column:\n", df.iloc[1]["input"])  # Print full input text
print("\nSummary Column:\n", df.iloc[1]["summary"])  # Print full summary

Input Column:
 To the Board of Directors and Shareowners of Alliant Energy Corporation Madison, Wisconsin We have audited the accompanying consolidated balance sheets of Alliant Energy Corporation and subsidiaries (the “Company”) as of December 31, 2016 and 2015, and the related consolidated statements of income, common equity, and cash flows for each of the three years in the period ended December 31, 2016. Our audits also included the Company’s financial statement schedules listed in the Index at These financial statements and financial statement schedules are the responsibility of the Company’s management. Our responsibility is to express an opinion on the financial statements and financial statement schedules based on our audits. We conducted our audits in accordance with the standards of the Public Company Accounting Oversight Board (United States). Those standards require that we plan and perform the audit to obtain reasonable assurance about whether the financial statements are 

on df.iloc[1] result =
---
Summary Column:
 I apologize, but the text you've provided appears to be a fragment of a financial statement with incomplete information. It seems to contain partial notes about revenue, profit/loss, and expenses, but lacks a clear, comprehensive summary. Without the full context and complete financial data, I cannot provide a meaningful summary.

To help you effectively, I would need:
1. The complete financial statement
2. Specific details about the company's financial performance
3. Full context of the revenue, expenses, and profit/loss figures

----

with is not good let try to improve it by using spaCy for  use spaCy for named entity recognition (NER) to extract key financial terms and numbers, then fill in the missing parts before passing the text to Claude for summarization.

In [ ]:
import anthropic
import pandas as pd
import re
import spacy

# Load English NLP model
nlp = spacy.load("en_core_web_sm")

df = pd.read_csv("first_3_rows.csv")
df["summary"] = ""

# Define save path
save_path = "test_claude_summary2.csv"

# Function to extract financial data dynamically
def extract_important_sections(text):
    sections = []

    # Extract numerical financial data
    revenue_match = re.search(r"(Total Revenue|Net Revenue|Operating Revenue).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    profit_match = re.search(r"(Net Income|Net Profit|Operating Income|Loss).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    expenses_match = re.search(r"(Operating Expenses|Total Expenses|Liabilities).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    assets_match = re.search(r"(Total Assets|Current Assets|Fixed Assets).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    liabilities_match = re.search(r"(Total Liabilities|Current Liabilities|Debt).*?(\d[\d,\.]*)", text, re.IGNORECASE)

    # Collect matches
    if revenue_match:
        sections.append("Revenue: " + revenue_match.group())
    if profit_match:
        sections.append("Profit/Loss: " + profit_match.group())
    if expenses_match:
        sections.append("Expenses: " + expenses_match.group())
    if assets_match:
        sections.append("Assets: " + assets_match.group())
    if liabilities_match:
        sections.append("Liabilities: " + liabilities_match.group())

    # If key financial data is found, return it
    if sections:
        return "\n".join(sections)

    # If no financial figures are found, use NLP to find relevant paragraphs
    doc = nlp(text)
    financial_keywords = ["earnings", "revenue", "profit", "loss", "expenses", "assets", "liabilities", "debt"]
    best_paragraph = ""

    for sent in doc.sents:
        # Check if sentence contains financial-related words
        if any(word in sent.text.lower() for word in financial_keywords):
            best_paragraph += sent.text + " "
            if len(best_paragraph) > 1000:  # Stop once enough content is collected
                break

    # If no relevant paragraph found, return first 1000 characters
    return best_paragraph.strip() if best_paragraph else text[:1000]

# Function to summarize text using Claude API
def summarize_text_claude(text):
    important_text = extract_important_sections(text)

    try:
        response = client.messages.create(
            model="claude-3-5-haiku-20241022",  # Cheaper Claude model
            max_tokens=200,
            temperature=0.3,
            messages=[{"role": "user", "content": f"Summarize this financial statement:\n\n{important_text}"}]
        )
        return response.content[0].text.strip()

    except anthropic.APIStatusError as e:
        print(f"Claude API error: {e}")
        return None

    except anthropic.RateLimitError as e:
        print(f"Claude rate limit exceeded: {e}. Waiting 60 seconds before retrying...")
        time.sleep(60)
        return summarize_text_claude(text)

    except anthropic.APIConnectionError as e:
        print(f"Claude API connection error: {e}. Retrying in 30 seconds...")
        time.sleep(30)
        return summarize_text_claude(text)

    except Exception as e:
        print(f"Unexpected error: {e}")
        return None



# Process and summarize each row
for i, row in df.iterrows():
    if pd.notna(row["summary"]) and row["summary"] != "":
        continue

    print(f"Processing row {i+1}/{len(df)} with Claude API...")

    summary = summarize_text_claude(row["input"])

    if summary is None:
        print("API error detected. Stopping to avoid data loss.")
        break

    df.at[i, "summary"] = summary

    # Save progress after each row
    df.to_csv(save_path, index=False)
    print(f"Progress saved at row {i+1}")

# Final save
df.to_csv(save_path, index=False)
print("\nSummarization completed. File saved as:", save_path)


Processing row 1/3 with Claude API...
Progress saved at row 1
Processing row 2/3 with Claude API...
Progress saved at row 2
Processing row 3/3 with Claude API...
Progress saved at row 3

Summarization completed. File saved as: test_claude_summary2.csv


see it's better

In [ ]:
print("Input Column:\n", df.iloc[0]["input"])  # Print full input text
print("\nSummary Column:\n", df.iloc[0]["summary"])  # Print full summary

Input Column:
 INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To the Board of Directors and Stockholders Digerati Technologies, Inc. San Antonio, Texas We have audited the accompanying consolidated balance sheets of Digerati Technologies, Inc. and its subsidiaries (collectively, “Digerati”) as of July 31, 2016 and July 31, 2015, and the related consolidated statements of operations, stockholders’ deficit and cash flows for the years then ended. These consolidated financial statements are the responsibility of Digerati’s management. Our responsibility is to express an opinion on these consolidated financial statements based on our audits. We conducted our audits in accordance with standards of the Public Company Accounting Oversight Board (United States). Those standards require that we plan and perform the audits to obtain reasonable assurance about whether the financial statements are free of material misstatement. Digerati is not required to have, nor were we engaged to perform, an audit

In [ ]:
print("Input Column:\n", df.iloc[1]["input"])  # Print full input text
print("\nSummary Column:\n", df.iloc[1]["summary"])  # Print full summary

Input Column:
 To the Board of Directors and Shareowners of Alliant Energy Corporation Madison, Wisconsin We have audited the accompanying consolidated balance sheets of Alliant Energy Corporation and subsidiaries (the “Company”) as of December 31, 2016 and 2015, and the related consolidated statements of income, common equity, and cash flows for each of the three years in the period ended December 31, 2016. Our audits also included the Company’s financial statement schedules listed in the Index at These financial statements and financial statement schedules are the responsibility of the Company’s management. Our responsibility is to express an opinion on the financial statements and financial statement schedules based on our audits. We conducted our audits in accordance with the standards of the Public Company Accounting Oversight Board (United States). Those standards require that we plan and perform the audit to obtain reasonable assurance about whether the financial statements are 

# GPT (70%) X Cluade (30%) 🔥

Now let's mix them together by:
- reads a CSV file containing financial statements
- assigns each row to be summarized by either ChatGPT or Claude,
- uses spaCy to  extract key financial sections
- generates a summary using the designated model.
- It continuously saves the progress after processing each row to avoid data loss.
- saves the completed summaries to a new CSV file.

test the library install:

In [ ]:
# check before using 15 usd credit
import openai
import anthropic
import pandas as pd
import os
import time
import re
import random
import spacy
from collections import defaultdict

print("openai version:", openai.__version__)
print("anthropic version:", anthropic.__version__)
print("pandas version:", pd.__version__)
print("spacy version:", spacy.__version__)


openai version: 1.61.0
anthropic version: 0.45.2
pandas version: 2.2.2
spacy version: 3.8.4


In [ ]:
CHATGPT_API_KEY =
CLAUDE_API_KEY =

debug non auto save :

In [ ]:
import os
import time
import random
import re
import pandas as pd
import spacy
import openai
import anthropic
from collections import defaultdict

# Initialize API clients
openai_client = openai.OpenAI(api_key=CHATGPT_API_KEY)
claude_client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

# Load the spaCy model for Natural Language Processing (NLP)
nlp = spacy.load("en_core_web_sm")
## --------------------------------

# Define file paths after dataset save but get interupt !
main_dataset = "main_dataset10k.csv"
save_path = "financial_statements_summary_10k_dataset.csv"

# Check if a saved progress file exists, otherwise start fresh
if os.path.exists(save_path):
    print("Resuming from saved progress...")

    # Load both the original dataset and saved progress
    df_original = pd.read_csv(main_dataset, encoding="utf-8", engine="python", on_bad_lines="skip")
    df_saved = pd.read_csv(save_path, encoding="utf-8", engine="python", on_bad_lines="skip")

    df = pd.merge(df_original, df_saved, on="input", how="left")

    # Ensure no duplicate rows
    df = df.drop_duplicates(subset=["input"]).reset_index(drop=True)

    # Print row counts to verify
    print(f"Rows in original dataset: {len(df_original)}")
    print(f"Rows in saved dataset: {len(df_saved)}")
    print(f"Total rows after merging (after removing duplicates): {len(df)}")  # Should be 10,000


    # Ensure missing values are handled correctly
    df = df.assign(
        summary=df["summary"].fillna(""),
        model=df["model"].fillna(df.apply(lambda _: "ChatGPT" if random.random() < 0.85 else "Claude", axis=1))
    )

    print(f"Total rows after merging: {len(df)}")
else:
    print("Starting fresh...")
    df = pd.read_csv(main_dataset, encoding="utf-8", engine="python", on_bad_lines="skip")
    df["summary"] = ""  # Initialize empty summaries
    df["model"] = df.apply(lambda _: "ChatGPT" if random.random() < 0.85 else "Claude", axis=1)

## --------------------------------

# # only run it at first time:
# # Define local save path
# local_path = "./"
# os.makedirs(local_path, exist_ok=True)

# save_path = os.path.join(local_path, "financial_statements_summary_10k_dataset.csv")

# # Keep only the required column
# df = df[["input"]].copy()

# # Load previous progress if available
# try:
#     df_saved = pd.read_csv(save_path)
#     df = df.merge(df_saved, on="input", how="left")  # Ensures all rows remain

#     # Fix: Use `assign()` instead of `fillna(inplace=True)`
#     df = df.assign(
#         summary=df["summary"].fillna(""),
#         model=df["model"].fillna(df.apply(lambda _: "ChatGPT" if random.random() < 0.85 else "Claude", axis=1))
#     )

#     print(f"Resumed from saved progress: {len(df_saved)} rows found.")
# except FileNotFoundError:
#     print("No previous save found, starting fresh.")

#     # Fix: Use direct assignment instead of `inplace=True`
#     df["summary"] = ""
#     df["model"] = df.apply(lambda _: "ChatGPT" if random.random() < 0.85 else "Claude", axis=1)

## --------------------------------

# Function to extract financial data dynamically
def extract_important_sections(text):
    sections = []

    # Extract numerical financial data using regex
    revenue_match = re.search(r"(Total Revenue|Net Revenue|Operating Revenue).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    profit_match = re.search(r"(Net Income|Net Profit|Operating Income|Loss).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    expenses_match = re.search(r"(Operating Expenses|Total Expenses|Liabilities).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    assets_match = re.search(r"(Total Assets|Current Assets|Fixed Assets).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    liabilities_match = re.search(r"(Total Liabilities|Current Liabilities|Debt).*?(\d[\d,\.]*)", text, re.IGNORECASE)

    # Collect matches
    if revenue_match:
        sections.append("Revenue: " + revenue_match.group())
    if profit_match:
        sections.append("Profit/Loss: " + profit_match.group())
    if expenses_match:
        sections.append("Expenses: " + expenses_match.group())
    if assets_match:
        sections.append("Assets: " + assets_match.group())
    if liabilities_match:
        sections.append("Liabilities: " + liabilities_match.group())

    # If key financial data is found, return it
    if sections:
        return "\n".join(sections)

    # If no financial figures are found, use NLP to find relevant paragraphs
    doc = nlp(text)
    financial_keywords = [
    # Core Financial Metrics
    "revenue", "profit", "loss", "expenses", "assets", "liabilities", "debt",

    # Performance & Investment Indicators
    "EBITDA", "ROI", "EPS", "profit margin", "operating margin",

    # Cash Flow & Liquidity
    "cash flow", "free cash flow", "working capital", "dividends",

    # Market & Valuation
    "P/E ratio", "market capitalization", "stock price"
    ]

    best_paragraph = ""

    for sent in doc.sents:
        # Check if sentence contains financial-related words
        if any(word in sent.text.lower() for word in financial_keywords):
            best_paragraph += sent.text + " "
            if len(best_paragraph) > 1000:  # Stop once enough content is collected
                break

    # If no relevant paragraph found, return first 1000 characters
    return best_paragraph.strip() if best_paragraph else text[:1000]

# Function to generate a structured summary with ChatGPT-3.5
def summarize_text_gpt(text):
    important_text = extract_important_sections(text)

    try:
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Provide a detailed summary of financial statements."},
                {"role": "user", "content": f"Summarize this financial statement in detail with specific sections:\n\n{important_text}"}
            ],
            max_tokens=300,
            temperature=0.3
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"ChatGPT Error: {e}")
        return None

# Function to summarize text using Claude API
def summarize_text_claude(text):
    important_text = extract_important_sections(text)
    word_count = len(important_text.split())  # Count words in input text

    # Auto-select model based on input length
    if word_count > 500:
        model_name = "claude-3-5-sonnet-20241022" # Use powerful model for long input
        max_tokens = 1000  # Allow more output for bigger model
    else:
        model_name = "claude-3-5-haiku-20241022"  # Use smaller, cheaper model
        max_tokens = 200  # Lower token limit for small model

    try:
        response = claude_client.messages.create(
            model=model_name,
            max_tokens=max_tokens,
            temperature=0.3,
            messages=[{"role": "user", "content": f"Summarize this financial statement:\n\n{important_text}"}]
        )
        return response.content[0].text.strip()

    except anthropic.APIStatusError as e:
        print(f"Claude API error: {e}")
        return "API ERROR: Unable to generate summary."

    except anthropic.RateLimitError as e:
        print(f"Claude rate limit exceeded: {e}. Waiting 60 seconds before retrying...")
        time.sleep(60)
        return summarize_text_claude(text)  # Retry after delay

    except anthropic.APIConnectionError as e:
        print(f"Claude API connection error: {e}. Retrying in 30 seconds...")
        time.sleep(30)
        return summarize_text_claude(text)  # Retry after delay

    except Exception as e:
        print(f"Unexpected error: {e}")
        return "ERROR: Summarization failed."


# Process and summarize
for i, row in df.iterrows():
    if pd.notna(row["summary"]) and row["summary"] != "":
        continue

    print(f"Processing row {i+1}/{len(df)} using {row['model']}...")

    if row["model"] == "ChatGPT":
        summary = summarize_text_gpt(row["input"])
    else:
        summary = summarize_text_claude(row["input"])

    if summary is None:
        print("API error detected. Stopping to avoid data loss.")
        break

    df.at[i, "summary"] = summary

    # Save progress after every row
    df.to_csv(save_path, index=False)
    print(f"Progress saved at row index {i} (total rows: {len(df)})")

# Final save
df.to_csv(save_path, index=False)
print("Summarization completed. File saved as:", save_path)

Resuming from saved progress...
Rows in original dataset: 7957
Rows in saved dataset: 7899
Total rows after merging (after removing duplicates): 7899
Total rows after merging: 7899
Processing row 6866/7899 using ChatGPT...
Progress saved at row index 6865 (total rows: 7899)
Processing row 6867/7899 using Claude...
Progress saved at row index 6866 (total rows: 7899)
Processing row 6868/7899 using ChatGPT...
Progress saved at row index 6867 (total rows: 7899)
Processing row 6869/7899 using ChatGPT...
Progress saved at row index 6868 (total rows: 7899)
Processing row 6870/7899 using ChatGPT...
Progress saved at row index 6869 (total rows: 7899)
Processing row 6871/7899 using Claude...
Progress saved at row index 6870 (total rows: 7899)
Processing row 6872/7899 using ChatGPT...
Progress saved at row index 6871 (total rows: 7899)
Processing row 6873/7899 using ChatGPT...
Progress saved at row index 6872 (total rows: 7899)
Processing row 6874/7899 using ChatGPT...
Progress saved at row index

check check check

In [ ]:
print("Summary Column:\n", df.iloc[3000]["model"])  # Print full summary
print("\nInput Column:\n", df.iloc[5000]["input"])  # Print full input text
print("\nSummary Column:\n", df.iloc[5000]["summary"])  # Print full summary

Summary Column:
 ChatGPT

Input Column:
 WUNONG ASIA PACIFIC COMPANY LIMITED INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To the Board of Directors and Shareholders of Wunong Asia Pacific Co. Ltd, Opinion on the Financial Statements We have audited the accompanying balance sheets of Wunong Asia Pacific Co. Ltd. (the “Company”) as of December 31, 2018 and 2017, and the related statements of operations, changes in stockholders’ deficits and statements of cash flows for the years then ended, and the related notes (collectively referred to as the financial statements). In our opinion, the financial statements referred to above present fairly, in all material respects, the financial positions of Wunong Asia Pacific Co. Ltd. as of December 31, 2018 and 2017, and the results of their operations and their cash flows for the years then ended in conformity with accounting principles generally accepted in the United States of America. Emphasis of Matter - Going Concern The accompanying financial st

### just check nothing important here:

In [ ]:
import pandas as pd
df = pd.read_csv("financial_statements_summary_10k_dataset_7899.csv")

In [ ]:
value_counts = df["model"].value_counts()

for value, count in value_counts.items():
    print(f"Value: {value}, Count: {count}")

In [ ]:
df.head()

,input,summary,model
0,INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To ...,The financial statement provided contains seve...,ChatGPT
1,. To the Board of Directors and Stockholders o...,The financial statement indicates that the com...,ChatGPT
2,. Report of Independent Registered Public Acco...,The financial statement provided includes deta...,ChatGPT
3,Index to Consolidated Financial Statements All...,The financial statement provided includes info...,ChatGPT
4,. To the Board of Directors and Stockholders R...,The financial statement provided includes info...,ChatGPT


In [ ]:
import pandas as pd

df_before = pd.read_csv("main_dataset10k.csv")
len(df_before)

10000

In [ ]:
import pandas as pd

df_after = pd.read_csv("financial_statements_summary_10k_dataset.csv")
len(df_after)

5160

In [ ]:
import pandas as pd

df_after = pd.read_csv("financial_statements_summary_10k_dataset_run5.csv")
len(df_after)

5380

In [ ]:
print("Summary Column:\n", df_after.iloc[2000]["model"])  # Print full summary
print("\nInput Column:\n", df_after.iloc[2000]["input"])  # Print full input text
print("\nSummary Column:\n", df_after.iloc[2000]["summary"])  # Print full summary

Summary Column:
 ChatGPT

Input Column:
 Report of Independent Registered Public Accounting Firm The Board of Directors and Stockholders of Hawkins, Inc.: We have audited the accompanying consolidated balance sheets of Hawkins, Inc. and subsidiaries (the Company) as of April 2, 2017 and April 3, 2016, and the related consolidated statements of income, comprehensive income, shareholders’ equity, and cash flows for each of the years in the three-year period ended April 2, 2017. In connection with our audits of the consolidated financial statements, we also have audited the financial statement schedule for each of the years in the three-year period ended April 2, 2017, listed in schedule II of this Form 10-K. We also have audited the Company’s internal control over financial reporting as of April 2, 2017, based on criteria established in Internal Control - Integrated Framework (2013) issued by the Committee of Sponsoring Organizations of the Treadway Commission (COSO). The Company’s manag

In [ ]:
import pandas as pd

df_after = pd.read_csv("financial_statements_summary_10k_dataset.csv")
len(df_after)

5099

In [ ]:
import pandas as pd

# Load the dataframes
# df_run_last = pd.read_csv("financial_statements_summary_10k_dataset_last_run.csv")
df_run1 = pd.read_csv("financial_statements_summary_10k_dataset.csv")
# df_run2 = pd.read_csv("financial_statements_summary_10k_dataset_run2.csv")
df_run3 = pd.read_csv("financial_statements_summary_10k_dataset_run10.csv")

# List of dataframes
dfs = [df_run1, df_run3]

for i, df in enumerate(dfs):
    non_nan_count = df['summary'].notna().sum()  # Count non-NaN values
    nan_count = df['summary'].isna().sum()  # Count NaN (empty) values
    print(f"DataFrame {i+1} - Non-NaN summaries: {non_nan_count}, Empty (NaN) summaries: {nan_count}")

DataFrame 1 - Non-NaN summaries: 6865, Empty (NaN) summaries: 1034
DataFrame 2 - Non-NaN summaries: 6865, Empty (NaN) summaries: 1034


In [ ]:
df.to_csv("financial_statements_summary_10k_dataset_run11.csv", index=False)

In [ ]:
df_after.head()

,input,summary,model
0,INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To ...,The financial statement provided contains seve...,ChatGPT
1,To the Board of Directors and Shareowners of A...,Here's a summary of the financial statement ex...,Claude
2,To the Board of Directors and Shareowners of A...,Based on the provided financial statement exce...,Claude
3,. To the Board of Directors and Stockholders o...,The financial statement indicates that the com...,ChatGPT
4,. Report of Independent Registered Public Acco...,NaN,ChatGPT


In [ ]:
df_check = pd.read_csv("financial_statements_summary_10k_dataset.csv", nrows=5)  # Read only first 5 rows
print(df_check.dtypes)  # Print column names and data types
print(df_check.head())  # Print first 5 rows to see the structure

input      object
summary    object
model      object
dtype: object
                                               input  \
0  INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To ...   
1  To the Board of Directors and Shareowners of A...   
2  To the Board of Directors and Shareowners of A...   
3  . To the Board of Directors and Stockholders o...   
4  . Report of Independent Registered Public Acco...   

                                             summary    model  
0  The financial statement provided contains seve...  ChatGPT  
1  Here's a summary of the financial statement ex...   Claude  
2  Based on the provided financial statement exce...   Claude  
3  The financial statement indicates that the com...  ChatGPT  
4                                                NaN  ChatGPT  


cannot resume debug

In [ ]:
import pandas as pd

# Read CSV while ensuring correct data types
df = pd.read_csv("financial_statements_summary_10k_dataset.csv", dtype={"input": str, "summary": str, "model": str})

# Display first 5 rows to confirm the format
print(df.head())

                                               input  \
0  INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To ...   
1  To the Board of Directors and Shareowners of A...   
2  To the Board of Directors and Shareowners of A...   
3  . To the Board of Directors and Stockholders o...   
4  . Report of Independent Registered Public Acco...   

                                             summary    model  
0  The financial statement provided contains seve...  ChatGPT  
1  Here's a summary of the financial statement ex...   Claude  
2  Based on the provided financial statement exce...   Claude  
3  The financial statement indicates that the com...  ChatGPT  
4                                                NaN  ChatGPT  


In [ ]:
df = df.assign(
    summary=df["summary"].fillna(""),
    model=df["model"].fillna("Unknown")
)

In [ ]:
df.head()

,input,summary,model
0,INDEX TO CONSOLIDATED FINANCIAL STATEMENTS To ...,The financial statement provided contains seve...,ChatGPT
1,To the Board of Directors and Shareowners of A...,Here's a summary of the financial statement ex...,Claude
2,To the Board of Directors and Shareowners of A...,Based on the provided financial statement exce...,Claude
3,. To the Board of Directors and Stockholders o...,The financial statement indicates that the com...,ChatGPT
4,. Report of Independent Registered Public Acco...,,ChatGPT


In [ ]:
len(df)

9372

# Only Claude (it's seem better than GPT):

for more 2k to make len main dataset = 10K

## show head()

In [ ]:
import pandas as pd

# Remove leading/trailing spaces in column names
df.columns = df.columns.str.strip()

# Rename 'cleaned_input' to 'input'
df = df.rename(columns={'cleaned_input': 'input'})

# Verify renaming worked
print(df.columns)
print(df.head())

Index(['input'], dtype='object')
                                               input
0  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...
1  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA  A...
2  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA.  ...
3  . Report of Independent Registered Public Acco...
4  . MGE Energy Management's Report on Internal C...


In [ ]:
len(df)

5000

In [ ]:
print(df['input'])

0       FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...
1       FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA  A...
2       FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA.  ...
3       . Report of Independent Registered Public Acco...
4       . MGE Energy Management's Report on Internal C...
                              ...                        
4995    Report of Independent Registered Public Accoun...
4996    FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA Re...
4997    MODEL N, INC. Index to Consolidated Financial ...
4998    INDEX TO CONSOLIDATED FINANCIAL STATEMENTS  AC...
4999    . Financial statement schedules other than tho...
Name: input, Length: 5000, dtype: object


In [ ]:
df.to_csv("extra_5000_cleaned_fixed.csv")

## code:

In [ ]:
CLAUDE_API_KEY =

In [ ]:
import os
import time
import random
import re
import pandas as pd
import spacy
import anthropic

# Initialize Claude API client
claude_client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

# Load the spaCy model for NLP
nlp = spacy.load("en_core_web_sm")

# Define file paths
main_dataset = "extra_5000_cleaned_fixed.csv"
save_path = "financial_statements_summary_5k_dataset.csv"

# Check if a saved progress file exists, otherwise start fresh
if os.path.exists(save_path):
    print("Resuming from saved progress...")
    df_original = pd.read_csv(main_dataset, encoding="utf-8", engine="python", on_bad_lines="skip")
    df_saved = pd.read_csv(save_path, encoding="utf-8", engine="python", on_bad_lines="skip")
    df = pd.merge(df_original, df_saved, on="input", how="left").drop_duplicates(subset=["input"]).reset_index(drop=True)
else:
    print("Starting fresh...")
    df = pd.read_csv(main_dataset, encoding="utf-8", engine="python", on_bad_lines="skip")
    df["summary"] = ""

# Ensure all rows use "Claude" as the model
df["model"] = "Claude"

# Function to extract financial data dynamically
def extract_important_sections(text):
    sections = []

    # Extract numerical financial data using regex
    revenue_match = re.search(r"(Total Revenue|Net Revenue|Operating Revenue).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    profit_match = re.search(r"(Net Income|Net Profit|Operating Income|Loss).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    expenses_match = re.search(r"(Operating Expenses|Total Expenses|Liabilities).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    assets_match = re.search(r"(Total Assets|Current Assets|Fixed Assets).*?(\d[\d,\.]*)", text, re.IGNORECASE)
    liabilities_match = re.search(r"(Total Liabilities|Current Liabilities|Debt).*?(\d[\d,\.]*)", text, re.IGNORECASE)

    # Collect matches
    if revenue_match:
        sections.append("Revenue: " + revenue_match.group())
    if profit_match:
        sections.append("Profit/Loss: " + profit_match.group())
    if expenses_match:
        sections.append("Expenses: " + expenses_match.group())
    if assets_match:
        sections.append("Assets: " + assets_match.group())
    if liabilities_match:
        sections.append("Liabilities: " + liabilities_match.group())

    # If key financial data is found, return it
    if sections:
        return "\n".join(sections)

    # If no financial figures are found, use NLP to find relevant paragraphs
    doc = nlp(text)
    financial_keywords = ["revenue", "profit", "loss", "expenses", "assets", "liabilities", "debt", "EBITDA",
                          "ROI", "EPS", "profit margin", "operating margin", "cash flow", "dividends", "stock price"]

    best_paragraph = ""
    for sent in doc.sents:
        if any(word in sent.text.lower() for word in financial_keywords):
            best_paragraph += sent.text + " "
            if len(best_paragraph) > 1000:
                break

    return best_paragraph.strip() if best_paragraph else text[:1000]

# Function to summarize text using Claude API
def summarize_text_claude(text):
    important_text = extract_important_sections(text)
    word_count = len(important_text.split())

    # Auto-select model based on input length
    if word_count > 500:
        model_name = "claude-3-5-sonnet-20241022"
        max_tokens = 1000
    else:
        model_name = "claude-3-5-haiku-20241022"
        max_tokens = 200

    try:
        response = claude_client.messages.create(
            model=model_name,
            max_tokens=max_tokens,
            temperature=0.3,
            messages=[{"role": "user", "content": f"Summarize this financial statement:\n\n{important_text}"}]
        )
        return response.content[0].text.strip()

    except anthropic.APIStatusError as e:
        print(f"Claude API error: {e}")
        return "API ERROR: Unable to generate summary."

    except anthropic.RateLimitError as e:
        print(f"Claude rate limit exceeded: {e}. Waiting 60 seconds before retrying...")
        time.sleep(60)
        return summarize_text_claude(text)

    except anthropic.APIConnectionError as e:
        print(f"Claude API connection error: {e}. Retrying in 30 seconds...")
        time.sleep(30)
        return summarize_text_claude(text)

    except Exception as e:
        print(f"Unexpected error: {e}")
        return "ERROR: Summarization failed."

# Process and summarize
for i, row in df.iterrows():
    if pd.notna(row["summary"]) and row["summary"] != "":
        continue

    print(f"Processing row {i+1}/{len(df)} using Claude...")

    summary = summarize_text_claude(row["input"])

    if summary is None:
        print("API error detected. Stopping to avoid data loss.")
        break

    df.at[i, "summary"] = summary

    # Save progress after every row
    df.to_csv(save_path, index=False)
    print(f"Progress saved at row index {i} (total rows: {len(df)})")

# Final save
df.to_csv(save_path, index=False)
print("Summarization completed. File saved as:", save_path)


Resuming from saved progress...
Processing row 734/4015 using Claude...
Progress saved at row index 733 (total rows: 4015)
Processing row 735/4015 using Claude...
Progress saved at row index 734 (total rows: 4015)
Processing row 736/4015 using Claude...
Progress saved at row index 735 (total rows: 4015)
Processing row 737/4015 using Claude...
Progress saved at row index 736 (total rows: 4015)
Processing row 738/4015 using Claude...
Progress saved at row index 737 (total rows: 4015)
Processing row 739/4015 using Claude...
Progress saved at row index 738 (total rows: 4015)
Processing row 740/4015 using Claude...
Progress saved at row index 739 (total rows: 4015)
Processing row 741/4015 using Claude...
Progress saved at row index 740 (total rows: 4015)
Processing row 742/4015 using Claude...
Progress saved at row index 741 (total rows: 4015)
Processing row 743/4015 using Claude...
Progress saved at row index 742 (total rows: 4015)
Processing row 744/4015 using Claude...
Progress saved at 

KeyboardInterrupt: 

In [ ]:
df.to_csv("5000_run3.csv")

In [ ]:
print("Summary Column:\n", df.iloc[500]["model"])  # Print full summary
print("\nInput Column:\n", df.iloc[500]["input"])  # Print full input text
print("\nSummary Column:\n", df.iloc[500]["summary"])  # Print full summary

Summary Column:
 Claude

Input Column:
 FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA - 16 - Report of Independent Registered Public Accounting Firm To the Board of Directors and Stockholders Repro Med Systems, Inc. Chester, New York We have audited the accompanying balance sheets of Repro Med Systems, Inc. as of February 29, 2016 and February 28, 2015, and the related statements of operations, stockholders’ equity and cash flows for the years then ended. These financial statements are the responsibility of the Company’s management. Our responsibility is to express an opinion on these financial statements based on our audits. We conducted our audits in accordance with the standards of the Public Company Accounting Oversight Board (United States). Those standards require that we plan and perform the audit to obtain reasonable assurance about whether the financial statements are free of material misstatement. The Company is not required to have, nor were we engaged to perform, an audit of 

In [ ]:
import pandas as pd

df_run1 = pd.read_csv("financial_statements_summary_5k_dataset.csv")
df_run2 = pd.read_csv("5000_run3.csv")

# List of dataframes
dfs = [df_run1, df_run2]

for i, df in enumerate(dfs):
    non_nan_count = df['summary'].notna().sum()  # Count non-NaN values
    nan_count = df['summary'].isna().sum()  # Count NaN (empty) values
    print(f"DataFrame {i+1} - Non-NaN summaries: {non_nan_count}, Empty (NaN) summaries: {nan_count}")

DataFrame 1 - Non-NaN summaries: 566, Empty (NaN) summaries: 0
DataFrame 2 - Non-NaN summaries: 2711, Empty (NaN) summaries: 1304


In [ ]:
len(df_run2)

4015

In [ ]:
df_run2_cleaned = df_run2.dropna(subset=['summary'])

len(df_run2_cleaned)

2711

In [ ]:
# Save cleaned datasets
df_run2_cleaned.to_csv("financial_statements_summary_5k_dataset.csv", index=False)

In [ ]:
import pandas as pd

df10k = pd.read_csv("financial_statements_summary_10k_dataset_7899.csv")
df5k = pd.read_csv("financial_statements_summary_5k_dataset.csv")

In [ ]:
# List of dataframes
dfs = [df5k, df10k]

for i, df in enumerate(dfs):
    non_nan_count = df['summary'].notna().sum()  # Count non-NaN values
    nan_count = df['summary'].isna().sum()  # Count NaN (empty) values
    print(f"DataFrame {i+1} - Non-NaN summaries: {non_nan_count}, Empty (NaN) summaries: {nan_count}")

DataFrame 1 - Non-NaN summaries: 2711, Empty (NaN) summaries: 0
DataFrame 2 - Non-NaN summaries: 7899, Empty (NaN) summaries: 0


In [ ]:
print(df5k.head())
print(df10k.head())

   Unnamed: 0  Unnamed: 0_x  \
0           0             0   
1           1             2   
2           2             3   
3           3             5   
4           4             6   

                                               input  Unnamed: 0.1  \
0  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...             0   
1  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA.  ...             1   
2  . Report of Independent Registered Public Acco...             2   
3  Index to Consolidated Financial Statements All...             3   
4  .  ACCOUNTING FIRM To the Board of Directors a...             4   

   Unnamed: 0_y                                            summary   model  
0             0  Here's a summary of the financial statement:\n...  Claude  
1             2  Based on the provided excerpt, here's a summar...  Claude  
2             3  This appears to be a partial financial stateme...  Claude  
3             5  This appears to be a partial financial stateme...  Claude  
4      

In [ ]:
df5k = df5k[['input', 'summary', 'model']]
df10k = df10k[['input', 'summary', 'model']]

df_merged = pd.concat([df5k, df10k], ignore_index=True)
df_merged.to_csv("Financial-EDGAR-CORPUS.csv", index=False)

print(df_merged.head())
print(f"Total merged rows: {len(df_merged)}")

                                               input  \
0  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA IN...   
1  FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA.  ...   
2  . Report of Independent Registered Public Acco...   
3  Index to Consolidated Financial Statements All...   
4  .  ACCOUNTING FIRM To the Board of Directors a...   

                                             summary   model  
0  Here's a summary of the financial statement:\n...  Claude  
1  Based on the provided excerpt, here's a summar...  Claude  
2  This appears to be a partial financial stateme...  Claude  
3  This appears to be a partial financial stateme...  Claude  
4  Here's a summary of the financial statement:\n...  Claude  
Total merged rows: 10610


In [ ]:
# Count unique values in the 'model' column
model_counts = df_merged['model'].value_counts()

# Print the counts
print(model_counts)

model
ChatGPT    6664
Claude     3946
Name: count, dtype: int64


In [ ]:
import pandas as pd
df = pd.read_csv('EDGAR-CORPUS-Financial-Summarization.csv')

In [ ]:
i = 8000
print("Summary Column:\n", df.iloc[i]["model"])
print("\nInput Column:\n", df.iloc[i]["input"])
print("\nSummary Column:\n", df.iloc[i]["summary"])

Summary Column:
 ChatGPT

Input Column:
 . To the Shareholders and Board of Directors of Exactus, Inc. Opinion on the Financial Statements We have audited the accompanying consolidated balance sheets of Exactus, Inc. (the “Company”), as of December 31, 2018 and 2017, and the related consolidated statements of operations, stockholders’ (deficit) equity and cash flows for each of the two years in the period ended December 31, 2018 and the related notes (collectively referred to as the “consolidated financial statements”). In our opinion, the consolidated financial statements present fairly, in all material respects, the financial position of the Company as of December 31, 2018 and 2017, and the results of its operations and its cash flows for each of the two years in the period ended December 31, 2018, in conformity with accounting principles generally accepted in the United States of America. The Company's Ability to Continue as a Going Concern The accompanying consolidated financial st